# A Basic Introduction

In the following introduction, we list some common use cases of `PineAPPL` with their solutions.

## Setting up

In order to start using `PineAPPL`, one first needs to install it. 
The easiest way to install the latest stable version is via `pip` 
using the following command:

In orderr to check that `PineAPPL` was installed properly, one can try to import it:

In [1]:
import pineappl

If the above command was successful, we can download the `PineAPPL` grid 
that will be used throughout this tutorial.

In [2]:
!wget --no-verbose --no-clobber 'https://data.nnpdf.science/pineappl/test-data/LHCB_DY_8TEV.pineappl.lz4'

We can now load the downloaded grid by running the following:

In [3]:
# Load the PineAPPL grid
grid = pineappl.grid.Grid.read("./LHCB_DY_8TEV.pineappl.lz4")

If the grid is actually a fast-kernel (FK) table, then it can loaded using the following:

```python
# Loading a FK table
fk = pineappl.fk_table.FkTable.read("path/to/fktable.pineappl.lz4")
```

**NOTE:** For the `pineappl.grid.Grid.read()` function, both `.pineappl` and `.pineappl.lz4` 
extensions are acceptable, as long as they are consistent. Without the `.lz4` extension, the
grid is assumed to not be compressed.

## How can I convolve a given PineAPPL grid with my PDF?

In [4]:
# We first need to load the PDF set with LHAPDF
import lhapdf

# Choose the PDF set to be used
pdfset = "NNPDF40_nnlo_as_01180"
# Use the central PDF, ie. member ID=0
pdf = lhapdf.mkPDF(pdfset, 0)

LHAPDF 6.5.4 loading /home/tanjona/LHAPDF_PATH/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331100


Our grid can now be convolved with our PDF set using the `convolve_with_one()` function:

In [5]:
predictions = grid.convolve_with_one(2212, pdf.xfxQ2, pdf.alphasQ2)
predictions.shape

(18,)

The length of `predictions` is the same as the number of bins that define the
observable in question. In our example, it is the inclusive cross-section for
DY production in bins of the muon pseudorapidity. See 
[arXiv:1511.08039](https://arxiv.org/abs/1511.08039) for more details.

**NOTE:** If the two initial state hadrons are different (as is the case in
$pp$ collisions in which one of the protons is polarized), then one can convolve
the grid with **two** different PDF sets using the `convolve_with_two()` function:

```python
# Convolve the two initial state hadrons with different PDF sets
predictions = grid.convolve_with_two(2212, polarized_pdf.xfxQ2, 2212, unpolarized_pdf.xfxQ2, unpolarized_pdf.alphasQ2)
```

**NOTE:** The same functions `convolve_with_one()` and `convolve_with_two()` also work for convolving FK tables with PDF sets.

## How can I get information on the contents of a given PineAPPL grid?

Once the `PineAPPL` grid is loaded, one can look into its contents and modify them.
Below we illustrate how to extract the most common information from a given grid.

In [6]:
# To get all the contributing channels
for idx, c in enumerate(grid.channels()):
    print(f"{idx}: {c}")

0: [(2, -2, 1.0), (4, -4, 1.0)]
1: [(0, -4, 1.0), (0, -2, 1.0)]
2: [(22, -4, 1.0), (22, -2, 1.0)]
3: [(2, 0, 1.0), (4, 0, 1.0)]
4: [(2, 22, 1.0), (4, 22, 1.0)]
5: [(1, -1, 1.0), (3, -3, 1.0)]
6: [(0, -3, 1.0), (0, -1, 1.0)]
7: [(22, -3, 1.0), (22, -1, 1.0)]
8: [(1, 0, 1.0), (3, 0, 1.0)]
9: [(1, 22, 1.0), (3, 22, 1.0)]
10: [(5, -5, 1.0)]
11: [(0, -5, 1.0)]
12: [(22, -5, 1.0)]
13: [(5, 0, 1.0)]
14: [(5, 22, 1.0)]
15: [(22, 22, 1.0)]
16: [(-5, 22, 1.0), (-3, 22, 1.0), (-1, 22, 1.0)]
17: [(1, 22, 1.0), (3, 22, 1.0), (5, 22, 1.0)]


In [7]:
# To get the information on the orders:
for idx, o in enumerate(grid.orders()):
    print(f"{idx}: {o.as_tuple()}")

0: (0, 2, 0, 0)
1: (1, 2, 0, 0)
2: (1, 2, 1, 0)
3: (1, 2, 0, 1)
4: (0, 3, 0, 0)
5: (0, 3, 1, 0)
6: (0, 3, 0, 1)


In [8]:
# To get the bin configurations
bin_dims = grid.bin_dimensions()

# Each element of bins is an object with a left and right limit and
# an associated bin normalization.
for idx, bin_dim in enumerate(range(bin_dims)):
    print(f"Bin configurations for index {idx}:")
    bin_left = grid.bin_left(0)
    bin_right = grid.bin_right(0)
    print(f"Left bins: {bin_left}")
    print(f"Right bins: {bin_right}")

Bin configurations for index 0:
Left bins: [2.    2.125 2.25  2.375 2.5   2.625 2.75  2.875 3.    3.125 3.25  3.375
 3.5   3.625 3.75  3.875 4.    4.25 ]
Right bins: [2.125 2.25  2.375 2.5   2.625 2.75  2.875 3.    3.125 3.25  3.375 3.5
 3.625 3.75  3.875 4.    4.25  4.5  ]


## How can I edit a grid?

The contents of PineAPPL grids can be edited in various ways. In our example,
we are going to change the normalization related to each bin.

Let us first check the normalizations before we mnodify them:

In [9]:
grid.bin_normalizations()

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125,
       0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.25 , 0.25 ])

In [10]:
import numpy as np

# Extract the left & right bin limits
bin_limits = [
    (left, right)
    for left, right in zip(
        grid.bin_left(bin_dims - 1), grid.bin_right(bin_dims - 1)
    )
]

# Multiply the normalization by a factor of `2`
normalizations = [2 * n for n in grid.bin_normalizations()]

# NOTE: `normalizations` have to be of type np.ndarray
remapper = pineappl.bin.BinRemapper(np.array(normalizations), bin_limits)

# Modify the bin normalization
grid.set_remapper(remapper)

# Save the modified grid
grid.write_lz4("./LHCB_DY_8TEV_custom_normalizations.pineappl.lz4")

We can now check that the normalizations have indeed been changed:

In [11]:
# Load our modified grids
grid_nrm = pineappl.grid.Grid.read("./LHCB_DY_8TEV_custom_normalizations.pineappl.lz4")
grid_nrm.bin_normalizations()

array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.5 , 0.5 ])